In [1]:
import geopandas as gpd
import pandas as pd
import classes.entropycalculator as ec
import shapely
from spatialentropy import altieri_entropy, leibovici_entropy
from scipy.stats import entropy
import numpy as np
import gc

from IPython.display import clear_output

from tqdm import tqdm
tqdm.pandas()

# select the part to handle
part = 2

In [2]:
gemeenten = gpd.read_parquet(f"data/wijken_parts/wijken_{part}.parquet")


In [3]:
def _get_shannon_entropy(labels, base=2):
    # get the total count of the labels
    total_count = len(labels)
    # get the unique labels and their counts
    _, label_counts = np.unique(labels, return_counts=True)

    probs = label_counts / total_count
    # get the entropy
    return entropy(probs, base=base)

def gm_total_amenities_entropy(gm_name, filter_i, area):
    L0_BLACKLIST, L1_BLACKLIST = ec.getfilter(filter_i)
    amenity_gdf = gpd.read_parquet(f"data/gm_amenities/amenities_{gm_name}.parquet")
    amenity_gdf = amenity_gdf[amenity_gdf.within(area)]
    
    # apply filters
    amenity_gdf = amenity_gdf[~amenity_gdf.L0_category.isin(L0_BLACKLIST)]
    if L1_BLACKLIST:
        for key, value in L1_BLACKLIST.items():
            amenity_gdf = amenity_gdf[
                ~(
                    (amenity_gdf.L0_category == key)
                    & (amenity_gdf.L1_category.isin(value))
                )
            ]
    
    # total number of amenities
    total_amenities = len(amenity_gdf)
    
    points = [[point.x, point.y] for point in amenity_gdf.geometry]
    
    # calculate entropy
    L0 = amenity_gdf.loc[:, f"L0_category"].values
    L1 = amenity_gdf.loc[:, f"L1_category"].values
    try:
        L0_entropy_lei = leibovici_entropy(points, L0, base=2).entropy
    except IndexError:
        L0_entropy_lei = 0
    try:
        L1_entropy_lei = leibovici_entropy(points, L1, base=2).entropy
    except IndexError:
        L1_entropy_lei = 0
    
    del points
    gc.collect()
    
    return total_amenities, L0_entropy_lei, L1_entropy_lei

In [4]:
for filter in [0, 1, 2]:
    print(f"Filter {filter}")
    for i, gm in tqdm(gemeenten.iterrows(), total=len(gemeenten)):
        total_amenities, L0_entropy_lei, L1_entropy_lei = gm_total_amenities_entropy(gm["gemeentenaam"], filter, gm["geometry"])
        # gemeenten.at[i, f"total_amenities_{filter}"] = total_amenities
        gemeenten.at[i, f"L0_leibovici_{filter}"] = L0_entropy_lei
        gemeenten.at[i, f"L1_leibovici_{filter}"] = L1_entropy_lei
        
        del total_amenities, L0_entropy_lei, L1_entropy_lei
        gc.collect()
    
gemeenten.to_parquet(f"data/wijken_parts/calculated/wijken_{part}.parquet")


Filter 0


100%|██████████| 416/416 [39:36<00:00,  5.71s/it]  


Filter 1


 14%|█▍        | 60/416 [04:34<27:10,  4.58s/it]  


KeyboardInterrupt: 